In [1]:
%load_ext line_profiler

In [161]:
import requests
import json

from IPython.display import Image
from IPython.core.display import HTML

import pandas as pd

import pprint
import mwparserfromhell
import re

In [ ]:
subject = 'Chùa'
url = 'https://vi.wikipedia.org/w/api.php'
params = {
        'action': 'query',
        'format': 'json',
        'titles': subject,
        'prop': 'extracts',
        'exintro': True,
        'explaintext': True,
    }

response = requests.get(url, params=params)
data = response.json()

page = next(iter(data['query']['pages'].values()))
print(page)

In [5]:
import requests
import pandas as pd

URL = "https://vi.wikipedia.org/w/api.php"
PARAMS = {
    "action": "query",
    "cmlimit": "100",
    "list": "categorymembers",
    "format": "json"
}

from IPython.display import clear_output
def getAllPages(SET_OF_CATS):
    data = {}
    
    i = 0
    for cat in SET_OF_CATS:
          # GET ALL PAGES
        DATA = S.get(url=URL, params={**PARAMS, "cmtitle": cat, "cmtype": "page"}).json()
        PAGES = DATA['query']['categorymembers']
        for page in PAGES:
            data[i] = {"page": page['title'], "cat": cat}
            i += 1
            print(f'Progress: {i}', end='\r')
            
    return pd.DataFrame.from_dict(data, orient='index')


def recurseCat(S, catset, title, level=0, maxlevel=5):
        # RECURSIVE
    R = S.get(url=URL, params={**PARAMS, "cmtitle": title, "cmtype": "subcat"})
    DATA = R.json()
    # print(DATA)
    PAGES = DATA['query']['categorymembers']

    index = 1
    for page in PAGES:
        print("- "*level,index, page['title'])
        catset.add(page['title'])
        
        if (level < maxlevel):
            recurseCat(S, catset, page['title'], level+1, maxlevel)
        else:
            print("Max level reached")
        index += 1

In [6]:
toptitle = "Thể_loại:Kiến_trúc_cổ_Việt_Nam"
S = requests.Session()
new_set = set()

recurseCat(S, new_set, toptitle,maxlevel=4)

 1 Thể loại:Tháp Chăm
-  1 Thể loại:Đền thờ Ấn Độ giáo tại Việt Nam
-  2 Thể loại:Đền thờ vua chúa Chăm Pa
 2 Thể loại:Chùa Việt Nam
-  1 Thể loại:Chùa Việt Nam theo tỉnh thành
- -  1 Thể loại:Chùa tại An Giang
- -  2 Thể loại:Chùa tại Bạc Liêu
- -  3 Thể loại:Chùa tại Bà Rịa – Vũng Tàu
- -  4 Thể loại:Chùa tại Bắc Giang
- -  5 Thể loại:Chùa tại Bắc Ninh
- -  6 Thể loại:Chùa tại Bến Tre
- -  7 Thể loại:Chùa tại Bình Định
- -  8 Thể loại:Chùa tại Bình Dương
- -  9 Thể loại:Chùa tại Bình Phước
- -  10 Thể loại:Chùa tại Bình Thuận
- -  11 Thể loại:Chùa tại Cần Thơ
- -  12 Thể loại:Chùa tại Cà Mau
- -  13 Thể loại:Chùa tại Đà Nẵng
- -  14 Thể loại:Chùa tại Đắk Lắk
- -  15 Thể loại:Chùa tại Đắk Nông
- -  16 Thể loại:Chùa tại Đồng Tháp
- -  17 Thể loại:Chùa tại Đồng Nai
- -  18 Thể loại:Chùa tại Gia Lai
- -  19 Thể loại:Chùa tại Hà Nam
- -  20 Thể loại:Chùa tại Hà Nội
- -  21 Thể loại:Chùa tại Hà Tĩnh
- -  22 Thể loại:Chùa tại Hải Dương
- -  23 Thể loại:Chùa tại Hải Phòng
- -  24 Thể loại:Ch

In [7]:
len(new_set)

340

In [8]:
s = list(new_set)
df = pd.DataFrame(s)
df.columns = ['test']

In [9]:
# df.test.str.split(':').str[1] \
#         .str.split(' ').str[:2].str.join(' ').value_counts()

In [14]:
substr = [
    'Di tích',
    'Chùa',
    'Đình',
    'Đền',
    'Miếu',
    'miếu',
    'Phố cổ',
    'Hoàng thành',
    'Lăng mộ',
    'Cung điện',
]
df = df[df.test.str.contains('|'.join(substr))].reset_index(drop=True)

In [15]:
all_pages = getAllPages(df.test)

In [16]:
all_pages

,page,cat
0,Khu di tích Pác Bó,Thể loại:Di tích tại Cao Bằng
1,Núi Khắc Thiệu,Thể loại:Di tích tại Cao Bằng
2,Thành Bản Phủ (Cao Bằng),Thể loại:Di tích tại Cao Bằng
3,Thành nhà Mạc (Phục Hòa),Thể loại:Di tích tại Cao Bằng
4,Chùa Khải Nam,Thể loại:Chùa tại Thanh Hóa
...,...,...
1468,Tử Cấm thành (Huế),Thể loại:Di tích Cố đô Huế
1469,Vạc đồng (nhà Nguyễn),Thể loại:Di tích Cố đô Huế
1470,Văn miếu (Huế),Thể loại:Di tích Cố đô Huế
1471,Viện Cơ mật (Huế),Thể loại:Di tích Cố đô Huế


In [17]:
df1 = all_pages
df2 = df1.groupby('page').size().reset_index(name='sum')
df2['sum'] = df2['sum'] - 1
df3 = pd.merge(df1, df2, on='page').groupby('cat')['sum'].sum().reset_index()
df3.sort_values(by='sum')

,cat,sum
140,Thể loại:Đình tại Hà Nam,0
45,Thể loại:Chùa tại Vĩnh Long,0
145,Thể loại:Đình tại Nghệ An,0
137,Thể loại:Đình tại Bến Tre,0
25,Thể loại:Chùa tại Lâm Đồng,0
...,...,...
79,Thể loại:Di tích tại Hà Nội,49
55,Thể loại:Di tích Cố đô Huế,68
158,Thể loại:Đền tại Việt Nam,80
62,Thể loại:Di tích quốc gia Việt Nam,114


In [18]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [19]:
allnodup = all_pages.drop_duplicates('page')
allnodup.sort_values('page')

,page,cat
599,An toàn khu II,Thể loại:Di tích quốc gia đặc biệt
508,An toàn khu Định Hóa,Thể loại:Di tích tại Thái Nguyên
250,Ao Bà Om,Thể loại:Di tích tại Trà Vinh
1016,Bia Lê Lợi,Thể loại:Di tích quốc gia Việt Nam
1133,Bia Lạc Giao,Thể loại:Di tích tại Đắk Lắk
...,...,...
1076,Động Tiên Phi,Thể loại:Di tích quốc gia Việt Nam
1352,Động Tiên Sơn (Lai Châu),Thể loại:Di tích tại Lai Châu
1077,Động Từ Thức,Thể loại:Di tích quốc gia Việt Nam
1078,Động Xá Nhè,Thể loại:Di tích quốc gia Việt Nam


In [162]:
def infobox_dict(url):
    response = requests.get(url)
    #Tập_tin:Chùa_Lý_Quốc_Sư.JPG
    data = response.json()

    # text = json.dumps(data, indent=2, ensure_ascii=False)
    # print(data['source'])
    # Image(url= "https://commons.wikimedia.org/wiki/Special:FilePath/Chùa Lý Quốc Sư.JPG", width=400)
    # print("no")

    revisions = data['source']

    if "{{Thông tin" not in revisions and "{{Infobox" not in revisions:
        return {}
        
    start = revisions.find("{{")
    end = revisions.find("\n}}", start)
    infobox = revisions[start:end]

    # Parse the infobox template into a dictionary
    d = {}
    for line in infobox.split("\n|"):
        if "=" in line:
            key, value = line.split("=", 1)
            val = value.strip()
            if bool(val):
                d[key.strip()] = val
            # d[key.strip()] = value.strip() #.replace('[','').replace(']','')

    # Print the resulting dictionary
    # pprint.pprint(d)
    return d

In [186]:
def tplinfobox(tpl):
    return tpl.name.contains('Thông tin') | tpl.name.contains('Infobox')

def infobox_dict_thing(url):
    response = requests.get(url)
    #Tập_tin:Chùa_Lý_Quốc_Sư.JPG
    data = response.json()

    # text = json.dumps(data, indent=2, ensure_ascii=False)
    # print(data['source'])
    # Image(url= "https://commons.wikimedia.org/wiki/Special:FilePath/Chùa Lý Quốc Sư.JPG", width=400)
    # print("no")

    revisions = data['parse']['wikitext']['*']
        
    # Parse the Wikitext using mwparserfromhell
    wikicode = mwparserfromhell.parse(revisions)

    # Find the infobox template
    templates = wikicode.filter_templates(matches=tplinfobox)
    if not templates:
        return {}
    
    # Extract the key-value pairs from the infobox
    d = {}
    for param in templates[0].params:
        val = str(param.value).strip()
        if bool(val):
            d[str(param.name).strip()] = val # Key

        # d[key.strip()] = value.strip() #.replace('[','').replace(']','')

    # Print the resulting dictionary
    # pprint.pprint(d)
    return d

In [187]:
def iterate_dict_infobox(url, list_of_loc, infoboxer):
    list_dict = []
    maxi = len(list_of_loc)
    i = 0
    for loc in list_of_loc:
        dict2 = {'page': loc, **infoboxer(url + loc)}
        list_dict.append(dict2)
        print(dict2)
        print(f'{i/maxi*100:.2f}%', end='\r')
        i += 1


    return list_dict

In [ ]:
url = 'https://vi.wikipedia.org/w/api.php?action=parse&prop=wikitext&format=json&page='
# %lprun -f iterate_dict_infobox iterate_dict_infobox(url, allnodup.page[0:20], infobox_dict_thing)
alldata = iterate_dict_infobox(url, allnodup.page, infobox_dict_thing)

{'page': 'Khu di tích Pác Bó'}
{'page': 'Núi Khắc Thiệu'}
{'page': 'Thành Bản Phủ (Cao Bằng)'}
{'page': 'Thành nhà Mạc (Phục Hòa)'}
{'page': 'Chùa Khải Nam'}
{'page': 'Chùa Sùng Nghiêm Diên Thánh'}
{'page': 'Dinh Hoàng A Tưởng'}
{'page': 'Đền Bảo Hà'}
{'page': 'Đền Cấm'}
{'page': 'Đền Mẫu Lào Cai'}
{'page': 'Đền Thượng (Lào Cai)'}
{'page': 'Chùa Đót Sơn'}
{'page': 'Chùa Hang (Đồ Sơn)'}
{'page': 'Chùa Phúc Lâm (Hải Phòng)'}
{'page': 'Chùa Trà Phương'}
{'page': 'Chùa Âng', 'tên': 'Chùa Âng <br> Angkorajaborey', 'hình': 'Chùa Âng.jpg', 'ghi chú hình': 'Chính điện chùa Âng', 'vĩ độ': '9', 'vĩ phút': '54', 'vĩ giây': '56.99', 'kinh độ': '106', 'kinh phút': '18', 'kinh giây': '12.78', 'quốc gia': '{{VIE}}', 'địa chỉ': '[[Quốc lộ 53]], khóm 4, [[Phường 8, thành phố Trà Vinh|phường 8]], thành phố [[Trà Vinh (thành phố)|Trà Vinh]], tỉnh [[Trà Vinh]]', 'tông phái': 'Phật giáo Nam Tông', 'khởi lập': '990', 'cấp di tích': 'quốc gia', 'loại di tích': 'Di tích văn hóa – lịch sử', 'ngày nhận': '{{sta

In [ ]:
alldata
# pd.DataFrame.from_dict(list_dict)

In [74]:
this = 'C'
print(this)
url = 'https://vi.wikipedia.org/w/rest.php/v1/page/' + this
%time infobox_dict(url)

Đền Đươi
CPU times: total: 219 ms
Wall time: 1.01 s


{}

In [185]:
this = 'Chùa Láng'
print(this)
url = 'https://vi.wikipedia.org/w/api.php?action=parse&prop=wikitext&format=json&page=' + this
%lprun -f infobox_dict_thing infobox_dict_thing(url)
# %time infobox_dict_thing(url)

Chùa Láng
Thông tin chùa
VNM
Tham khảo
Webarchive
Chùa tại Việt Nam
Du lịch Hà Nội


Timer unit: 1e-07 s

Total time: 0.725976 s
File: C:\Users\Legion\AppData\Local\Temp\ipykernel_24792\626791405.py
Function: infobox_dict_thing at line 5

Line #      Hits         Time  Per Hit   % Time  Line Contents
     5                                           def infobox_dict_thing(url):
     6         1    7181092.0 7181092.0     98.9      response = requests.get(url)
     7                                               #Tập_tin:Chùa_Lý_Quốc_Sư.JPG
     8         1        822.0    822.0      0.0      data = response.json()
     9                                           
    10                                               # text = json.dumps(data, indent=2, ensure_ascii=False)
    11                                               # print(data['source'])
    12                                               # Image(url= "https://commons.wikimedia.org/wiki/Special:FilePath/Chùa Lý Quốc Sư.JPG", width=400)
    13                                               # print("no")
    14   

In [172]:
infobox_dict_thing(url)

{}

In [4]:
list_dict = []
list_chua = ['Lăng_Cha_Cả']

for loc in list_chua: 
    list_dict.append(infobox_dict('https://vi.wikipedia.org/w/rest.php/v1/page/' + loc))

list_dict

[{}]

In [78]:
response = requests.get('https://vi.wikipedia.org/w/rest.php/v1/page/html/Đền_Đươi')
response.json()

{'messageTranslations': {'vi': 'The requested relative path (/v1/page/html/%C4%90%E1%BB%81n_%C4%90%C6%B0%C6%A1i) did not match any known handler',
  'en': 'The requested relative path (/v1/page/html/%C4%90%E1%BB%81n_%C4%90%C6%B0%C6%A1i) did not match any known handler'},
 'httpCode': 404,
 'httpReason': 'Not Found'}